<a href="https://colab.research.google.com/github/kartoch/colab-eda/blob/master/98%20-%20Loading%20and%20Saving%20from%20GCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.oauth2 import service_account
from google.cloud.storage import client
import io
from io import BytesIO
from zipfile import ZipFile

In [0]:
key_path = "datascience-265115-74e8e66e6df1.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = client.Client(
    credentials=credentials,
    project=credentials.project_id,
)

In [0]:
START_YEAR = 2007
END_YEAR = 2018
BUCKET = "eda-essence-vincent"

In [0]:
def download_datasets(year, bucket):
    blob_pathname = "PrixCarburants_annuel_" + str(year) + ".zip"
    blob = bucket.blob(blob_pathname)
    return ZipFile(io.BytesIO(blob.download_as_string()))

def generator_zip_file(client):
    bucket = client.bucket("essence-dataset-eda")
    for year in range(START_YEAR,END_YEAR+1):
        zip_ref = download_datasets(year, bucket)    
        [xml_filename] = zip_ref.namelist()
        yield (zip_ref.open(xml_filename),year)
        zip_ref.close()

In [70]:
for f,year in generator_zip_file(client):
    print(f,year)

<zipfile.ZipExtFile name='PrixCarburants_annuel_2007.xml' mode='r' compress_type=deflate> 2007
<zipfile.ZipExtFile name='PrixCarburants_annuel_2008.xml' mode='r' compress_type=deflate> 2008
<zipfile.ZipExtFile name='PrixCarburants_annuel_2009.xml' mode='r' compress_type=deflate> 2009
<zipfile.ZipExtFile name='PrixCarburants_annuel_2010.xml' mode='r' compress_type=deflate> 2010
<zipfile.ZipExtFile name='PrixCarburants_annuel_2011.xml' mode='r' compress_type=deflate> 2011
<zipfile.ZipExtFile name='PrixCarburants_annuel_2012.xml' mode='r' compress_type=deflate> 2012
<zipfile.ZipExtFile name='PrixCarburants_annuel_2013.xml' mode='r' compress_type=deflate> 2013
<zipfile.ZipExtFile name='PrixCarburants_annuel_2014.xml' mode='r' compress_type=deflate> 2014
<zipfile.ZipExtFile name='PrixCarburants_annuel_2015.xml' mode='r' compress_type=deflate> 2015
<zipfile.ZipExtFile name='PrixCarburants_annuel_2016.xml' mode='r' compress_type=deflate> 2016
<zipfile.ZipExtFile name='PrixCarburants_annuel_20

In [0]:
def save_file():
    bucket = client.get_bucket(BUCKET)
    with open("/tmp/test.txt", "w+") as f:
        blob = bucket.blob("test.txt")
        blob.upload_from_filename("/tmp/test.txt")

In [0]:
save_file()